In [243]:
import os
import re
import glob
import difflib
import pandas as pd
import numpy as np
import itertools

In [244]:
path = 'C:/analyticsdev/Projects/EXPECTEDX/Alloe/data'
files = glob.glob(os.path.join(path, "*.csv"))

raw_data = [pd.read_csv(name) for name in files]
df_names = [re.findall('(?<=\.)[a-zA-Z]+(?=\.)', names) for names in files]
df_names = itertools.chain(*df_names)
raw_data = dict(zip(df_names, raw_data))

In [245]:
answer = raw_data['answer']
answer['survey'] = answer['survey'].apply(lambda x: re.split(' (?=@)',x)[0])
answer['user'] = answer['user'].apply(lambda x: re.split(' (?=@)',x)[0])

In [246]:
answer = answer.pivot_table(index='user', 
                            columns='survey', 
                            values='answer', 
                            aggfunc=lambda x: ' '.join(str(v) for v in x))
answer = answer.add_suffix('_surv')

In [247]:
main = raw_data['user'].merge(answer, 
                              how='left', 
                              left_on='_id', 
                              right_index=True, 
                              sort=True)

In [248]:
chal_inv = raw_data['challengeInvitation']
chal_inv['challenge'] = chal_inv['challenge'].apply(lambda x: re.split(' (?=@)',x)[0])
chal_inv['friend'] = chal_inv['friend'].apply(lambda x: re.split(' (?=@)',x)[0])
chal_inv['user'] = chal_inv['user'].apply(lambda x: re.split(' (?=@)',x)[0])

In [249]:
chal_inv = chal_inv.pivot_table(index='user',
                                columns='challenge', 
                                values='friend',
                                aggfunc=lambda x: ' '.join(str(v) for v in x))

In [250]:
chal_inv = chal_inv.add_suffix('_chalinv')

In [251]:
main = main.merge(chal_inv,
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

Don't understand relationships between "challenger" table and "challengeInvitation" table 'user' column

In [252]:
frnd_rq = raw_data['friendship']
frnd_rq['friend'] = frnd_rq['friend'].apply(lambda x: re.split(' (?=@)',x)[0])
frnd_rq['user'] = frnd_rq['user'].apply(lambda x: re.split(' (?=@)',x)[0])

In [253]:
frnd_rq = frnd_rq.pivot_table(index='user',
                              columns='friend',
                              values='status',
                              aggfunc=lambda x: ' '.join(str(v) for v in x))
frnd_rq = frnd_rq.add_suffix('_friendid')

In [254]:
main = main.merge(frnd_rq,
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

In [255]:
event_ast = raw_data['assistantEvent']

In [256]:
event_ast
event_ast['event'] = event_ast['event'].apply(lambda x: re.split(' (?=@)',x)[0])
event_ast['user'] = event_ast['user'].apply(lambda x: re.split(' (?=@)',x)[0])

In [257]:
event_ast = event_ast.pivot_table(index='user',
                                  columns='event',
                                  values='_id',
                                  aggfunc=lambda x: ' '.join(str(v) for v in x))
event_ast = event_ast.add_suffix('_eventid')

In [258]:
main = main.merge(event_ast,
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

In [259]:
post = raw_data['post']
post['user'] = post['user'].apply(lambda x: re.split(' (?=@)',x)[0])

In [260]:
post['counter'] = int(1)
post = post.pivot_table(index='user',
                        columns='item._class',
                        values='counter',
                        aggfunc='sum')
post = post.add_suffix('_posttype')

In [261]:
main = main.merge(post,
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

In [267]:
metrics_miles_time = raw_data['post'][['user','item.miles','item.time','item.positions']]
metrics_miles_time = metrics_miles_time.groupby(metrics_miles_time.user).sum()

metrics_exercise = raw_data['post'][['user','item.exercise']]
metrics_exercise['counter'] = int(1)

In [268]:
metrics_exercise = metrics_exercise.pivot_table(index='user',
                                                columns='item.exercise',
                                                values='counter',
                                                aggfunc='sum')

In [269]:
main = main.merge(metrics_miles_time,
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

main = main.merge(metrics_exercise,
                  how='left',
                  left_on='_id',
                  right_index=True,
                  sort=True)

In [271]:
main

_class                       _id active  \
185  com.litekey.alloe.model.User  54f530089932b01dbcffc89b   True   
134  com.litekey.alloe.model.User  54f530c89932b01dbcffc89c   True   
110  com.litekey.alloe.model.User  54f5ef739932b01dbcffc8a2   True   
198  com.litekey.alloe.model.User  54f757f4993241fc498ec5d7   True   
228  com.litekey.alloe.model.User  54f8f6f59932ce1b71e7317a   True   
43   com.litekey.alloe.model.User  54fe33869932ce1b71e73193   True   
53   com.litekey.alloe.model.User  54fe49989932ce1b71e73194   True   
0    com.litekey.alloe.model.User  5509a55b993212d9d6de6e35   True   
315  com.litekey.alloe.model.User  550a054e993212d9d6de6e3e   True   
15   com.litekey.alloe.model.User  550a1875993212d9d6de6e4e   True   
446  com.litekey.alloe.model.User  550a6c30993212d9d6de6e63   True   
21   com.litekey.alloe.model.User  550af727993212d9d6de6e76   True   
44   com.litekey.alloe.model.User  550b0e7a993212d9d6de6e86   True   
20   com.litekey.alloe.model.User  550b1631993212d9d6de6e8f   True   
161  com.litekey.alloe.model.User  550b1aa8993212d9d6de6ea6   True   
1    com.litekey.alloe.model.User  550b1ad0993212d9d6de6ea7   True   
34   com.litekey.alloe.model.User  550b1b4f993212d9d6de6ee9   True   
239  com.litekey.alloe.model.User  550b1c9f993212d9d6de6f05   True   
2    com.litekey.alloe.model.User  550b1d3c993212d9d6de6f06   True   
23   com.litekey.alloe.model.User  550b1e77993212d9d6de6f15   True   
480  com.litekey.alloe.model.User  550b1ebd993212d9d6de6f16   True   
18   com.litekey.alloe.model.User  550b1f6a993212d9d6de6f1a   True   
19   com.litekey.alloe.model.User  550b2a66993212d9d6de6f51   True   
29   com.litekey.alloe.model.User  550b2e0a993212d9d6de6f69   True   
85   com.litekey.alloe.model.User  550b2e45993212d9d6de6f6a   True   
337  com.litekey.alloe.model.User  550b2ff0993212d9d6de6fba   True   
4    com.litekey.alloe.model.User  550b30da993212d9d6de6ffa   True   
208  com.litekey.alloe.model.User  550b37db993212d9d6de70b2   True   
24   com.litekey.alloe.model.User  550b410c993212d9d6de70c5   True   
222  com.litekey.alloe.model.User  550b56bd993212d9d6de70e2   True   
..                            ...                       ...    ...   
563  com.litekey.alloe.model.User  5981078c0cf21a1264c92d03   True   
564  com.litekey.alloe.model.User  5981384e0cf21a1264c92e86   True   
365  com.litekey.alloe.model.User  5981fcfb0cf21a1264c931e2   True   
566  com.litekey.alloe.model.User  5982517b0cf21a1264c9326b   True   
366  com.litekey.alloe.model.User  59876a000cf2f6a709cd0f9d   True   
568  com.litekey.alloe.model.User  598c9a530cf23ac860c2ed50   True   
569  com.litekey.alloe.model.User  59923a4b0cf23ac860c2f844   True   
571  com.litekey.alloe.model.User  5994bcf30cf23ac860c30002   True   
572  com.litekey.alloe.model.User  5994c0fd0cf23ac860c3001a   True   
573  com.litekey.alloe.model.User  5994c9610cf23ac860c3005a   True   
574  com.litekey.alloe.model.User  599b4b4f0cf23ac860c30d6b   True   
367  com.litekey.alloe.model.User  599cac300cf23ac860c31770   True   
576  com.litekey.alloe.model.User  59a0b6ec0cf23ac860c32335   True   
368  com.litekey.alloe.model.User  59a18d670cf23ac860c323f9   True   
577  com.litekey.alloe.model.User  59a717340cf294679b36f58d   True   
578  com.litekey.alloe.model.User  59a8afc70cf294679b36f86a   True   
579  com.litekey.alloe.model.User  59ab1ce30cf294679b36fb4c   True   
580  com.litekey.alloe.model.User  59aef84e0cf294679b36ffb7   True   
581  com.litekey.alloe.model.User  59b8bc370cf2b2415f79e1d7   True   
583  com.litekey.alloe.model.User  59c048e70cf20846b04ef934   True   
584  com.litekey.alloe.model.User  59c3373e0cf20846b04f0179   True   
370  com.litekey.alloe.model.User  59c540f40cf20846b04f06fe   True   
585  com.litekey.alloe.model.User  59cae5610cf20846b04f11b5   True   
586  com.litekey.alloe.model.User  59cd379d0cf20846b04f17b1   True   
587  com.litekey.alloe.model.User  59ce7e410cf20846b04f1a43   True   
588  com.litekey.alloe.model.User  59